In [1]:
#load packages
import pandas as pd 
import numpy as np
from geopy import distance
from geopy import Point
import fuzzymatcher
pd.set_option('display.max_columns', 999)

In [2]:
# load FW datasets
fw = pd.read_csv("..//Datasets/NEW/FW_flights.csv")
fw_info = pd.read_csv("..//Datasets/NEW/FW_services_info.csv")

# load airport data
airports = pd.read_csv("..//Datasets/Airports/BTS_Airports_LAT_LON.csv")
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

In [3]:
fw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23160 entries, 0 to 23159
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tail_number           23160 non-null  object
 1   date                  23160 non-null  object
 2   aircraft              23160 non-null  object
 3   origin                23160 non-null  object
 4   origin_location       23160 non-null  object
 5   destination           23159 non-null  object
 6   destination_location  23160 non-null  object
 7   departure             23160 non-null  object
 8   arrival               22734 non-null  object
 9   duration              23148 non-null  object
dtypes: object(10)
memory usage: 1.8+ MB


In [4]:
fw_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Tail_number             333 non-null    object
 1   Summary                 329 non-null    object
 2   Owner                   329 non-null    object
 3   Airworthiness_Class     329 non-null    object
 4   Serial_Number           329 non-null    object
 5   Engine                  329 non-null    object
 6   Weight                  329 non-null    object
 7   Speed                   329 non-null    object
 8   Mode_S_Code             329 non-null    object
 9   Status                  329 non-null    object
 10  Certificate_Issue_Date  322 non-null    object
 11  Airworthiness_Date      312 non-null    object
 12  Last_Action_Date        329 non-null    object
 13  Expiration              322 non-null    object
 14  Registry_Source         329 non-null    object
dtypes: obj

In [5]:
# add Owner columns
fw = fw.merge(fw_info[['Tail_number','Owner']], how='left', left_on='tail_number', right_on='Tail_number').drop('Tail_number', axis=1)

In [6]:
# Remove useless words
fw = fw.replace(regex={'Near ':'', 'First seen ':'', 'Last seen ':'', 'En Route':None, 'Unknown':None, 'Diverted':None, '¬†':' '})

In [7]:
# check for duplicates
fw.duplicated().sum()

0

In [8]:
# check for null values
fw.isna().sum()

tail_number               0
date                      0
aircraft                194
origin                    0
origin_location           0
destination               1
destination_location      1
departure                 0
arrival                 426
duration                449
Owner                     0
dtype: int64

In [9]:
# split origin_Latitude and origin_Longitude
fw['origin_Latitude'] = fw[fw.origin_location.str.startswith('L ')].origin_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[0]
fw['origin_Longitude'] = fw[fw.origin_location.str.startswith('L ')].origin_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[1]

In [10]:
# split destination_Latitude and destination_Longitude
fw['destination_Latitude'] = fw[fw.destination_location.str.startswith('L ', na=False)].destination_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[0]
fw['destination_Longitude'] = fw[fw.destination_location.str.startswith('L ', na=False)].destination_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[1]

In [11]:
# convert to numeric
fw['origin_Latitude'] = pd.to_numeric(fw.origin_Latitude, errors='coerce')
fw['origin_Longitude'] = pd.to_numeric(fw.origin_Longitude, errors='coerce')

fw['destination_Latitude'] = pd.to_numeric(fw.destination_Latitude, errors='coerce')
fw['destination_Longitude'] = pd.to_numeric(fw.destination_Longitude, errors='coerce')

In [12]:
# split ICAO_code
fw['ICAO_code_origin'] = fw[fw.origin_Latitude.isna()].origin_location.str.rsplit(" - ", n = 1, expand = True)[1]
fw['ICAO_code_destination'] = fw[fw.destination_Latitude.isna()].destination_location.str.rsplit(" - ", n = 1, expand = True)[1]

In [13]:
# replace dual ICAO code
fw['spare'] = fw.ICAO_code_origin.str.split(" / ", expand=True, n=1)[1]
fw.loc[:,'ICAO_code_origin'] = fw.ICAO_code_origin.str.split(" / ", expand=True, n=1)[0]
fw.loc[fw.spare.notnull(),'ICAO_code_origin'] = fw.loc[fw.spare.notnull(), 'spare']
fw = fw.drop('spare', axis=1)

fw['spare'] = fw.ICAO_code_destination.str.split(" / ", expand=True, n=1)[1]
fw.loc[:,'ICAO_code_destination'] = fw.ICAO_code_destination.str.split(" / ", expand=True, n=1)[0]
fw.loc[fw.spare.notnull(),'ICAO_code_destination'] = fw.loc[fw.spare.notnull(), 'spare']
fw = fw.drop('spare', axis=1)

In [14]:
# check duplicates airport_codes
airport_codes.duplicated(subset=['ident']).sum()

0

In [15]:
# split coordinates and convert to numeric
airport_codes["Latitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[1])
airport_codes["Longitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[0])

In [16]:
# merge origin airport information
fw = fw.merge(airport_codes[['ident', 'iata_code', 'Latitude', 'Longitude']].add_suffix('_origin'),
         how='left', left_on='ICAO_code_origin', right_on='ident_origin')

In [17]:
# merge destination airport information
fw = fw.merge(airport_codes[['ident', 'iata_code', 'Latitude', 'Longitude']].add_suffix('_destination'),
         how='left', left_on='ICAO_code_destination', right_on='ident_destination')

In [18]:
# merge the same 2 columns
fw.origin_Latitude = fw.origin_Latitude.fillna(0) + fw.Latitude_origin.fillna(0)
fw.origin_Longitude = fw.origin_Longitude.fillna(0) + fw.Longitude_origin.fillna(0)

fw.destination_Latitude = fw.destination_Latitude.fillna(0) + fw.Latitude_destination.fillna(0)
fw.destination_Longitude = fw.destination_Longitude.fillna(0) + fw.Longitude_destination.fillna(0)

In [19]:
# drop duplicated columns
fw = fw.drop(['ident_origin', 'ident_destination', 'Latitude_origin', 'Longitude_origin', 'Latitude_destination', 'Longitude_destination'], axis=1)

In [20]:
# replace abbreviations to full name for the best matching
fw = fw.replace(regex={r'Muni': 'Municipal', r'Rgnl':'Regional', r'Intl':'International', r"Int'l":'International',
                 r'Trml':'Terminal', r'Fld':'Field'})

In [21]:
# split states for the best matching
fw['origin_state'] = fw.origin.str.split(', ', n=1, expand=True)[1]
fw.loc[fw.origin_state.isna(), 'origin_state'] = fw.origin_location.str.split(', ', n=1, expand=True)[1].str.rsplit(') ', n=1, expand=True)[0]

fw['destination_state'] = fw.destination.str.split(', ', n=1, expand=True)[1]
fw.loc[fw.destination_state.isna(), 'destination_state'] = fw.destination_location.str.split(', ', n=1, expand=True)[1].str.rsplit(') ', n=1, expand=True)[0]

In [22]:
# replace location back to NA for the best matching
fw.loc[fw.origin_Latitude == 0, 'origin_Latitude'] = np.nan
fw.loc[fw.origin_Longitude == 0, 'origin_Longitude'] = np.nan

fw.loc[fw.destination_Latitude == 0, 'destination_Latitude'] = np.nan
fw.loc[fw.destination_Longitude == 0, 'destination_Longitude'] = np.nan

In [23]:
# split cities for the best matching
fw['city_origin'] = fw.origin_location.str.rsplit(" (", n=1, expand=True)[1].str.split(")", n=1, expand=True)[0].str.replace(', ', ' - ')
fw.loc[fw.city_origin.isna(), 'city_origin'] = fw.loc[fw.city_origin.isna(), 'origin'].str.replace(', ', ' - ')

fw['city_destination'] = fw.destination_location.str.rsplit(" (", n=1, expand=True)[1].str.split(")", n=1, expand=True)[0].str.replace(', ', ' - ')
fw.loc[fw.city_destination.isna(), 'city_destination'] = fw.loc[fw.city_destination.isna(), 'destination'].str.replace(', ', ' - ')

In [24]:
fw.isna().sum()

tail_number                 0
date                        0
aircraft                  194
origin                      0
origin_location             0
destination                 1
destination_location        1
departure                   0
arrival                   426
duration                  449
Owner                       0
origin_Latitude           666
origin_Longitude          666
destination_Latitude      626
destination_Longitude     626
ICAO_code_origin          361
ICAO_code_destination     545
iata_code_origin         2771
iata_code_destination    2828
origin_state               35
destination_state          36
city_origin                 0
city_destination            1
dtype: int64

In [25]:
# drop duplicates
airports = airports.drop_duplicates(subset=['AIRPORT_CODE'])

In [26]:
# clean CITY_NAME columns
airports.loc[airports.CITY_NAME.str.contains('/'), 'CITY_NAME'] = airports.loc[airports.CITY_NAME.str.contains('/'), 'CITY_NAME']\
                                                                          .str.split("/", n=1, expand=True)[0] + ' - ' + airports\
         .loc[airports.CITY_NAME.str.contains('/'), 'CITY_NAME'].str.split("/", n=1, expand=True)[1].str.split(" - ", n=1, expand=True)[1]

In [27]:
# fuzzy merge on locations with origin airports
fw = fuzzymatcher.fuzzy_left_join(fw, airports.add_suffix('_origin'), 
                                  left_on = ['origin_state', 'city_origin', 'origin_Latitude', 'origin_Longitude', 'origin'], 
                                  right_on = ['STATE_CODE_origin', 'CITY_NAME_origin','LATITUDE_origin', 'LONGITUDE_origin',
                                              'AIRPORT_NAME_origin']).iloc[:,3:].reset_index(drop=True)

In [28]:
# drop origin_state mismatching
fw.loc[(fw.origin_state != fw.STATE_CODE_origin) & (fw.origin_state.notnull()), 
       ['AIRPORT_CODE_origin', 'AIRPORT_NAME_origin', 'CITY_NAME_origin', 'COUNTRY_NAME_origin', 
        'STATE_NAME_origin', 'STATE_CODE_origin', 'LATITUDE_origin', 'LONGITUDE_origin']] = np.nan

In [29]:
# drop origin_Longitude mistachings
fw.loc[(fw.origin_Longitude < 0) & (fw.LONGITUDE_origin > 0),
    ['AIRPORT_CODE_origin', 'AIRPORT_NAME_origin', 'CITY_NAME_origin', 'COUNTRY_NAME_origin', 
        'STATE_NAME_origin', 'STATE_CODE_origin', 'LATITUDE_origin', 'LONGITUDE_origin']] = np.nan

In [30]:
# drop city_origin mismatching
fw.loc[fw.city_origin != fw.CITY_NAME_origin,
    ['AIRPORT_CODE_origin', 'AIRPORT_NAME_origin', 'CITY_NAME_origin', 'COUNTRY_NAME_origin', 
        'STATE_NAME_origin', 'STATE_CODE_origin', 'LATITUDE_origin', 'LONGITUDE_origin']] = np.nan

In [31]:
# replace null values in iata_code_origin
fw.loc[fw.iata_code_origin.isna(),'iata_code_origin']  = fw.loc[fw.iata_code_origin.isna(), 'AIRPORT_CODE_origin']

In [32]:
# fuzzy merge on locations with destination airports
fw = fuzzymatcher.fuzzy_left_join(fw, airports.add_suffix('_destination'), left_on = ['destination_state', 'city_destination', 
                                                                                      'destination_Latitude', 'destination_Longitude', 'destination'], 
                                  right_on = ['STATE_CODE_destination', 'CITY_NAME_destination', 'LATITUDE_destination', 'LONGITUDE_destination',
                                              'AIRPORT_NAME_destination']).iloc[:,3:].reset_index(drop=True)

In [33]:
# drop destination_state mismatching
fw.loc[(fw.destination_state != fw.STATE_CODE_destination) & (fw.STATE_CODE_destination.notnull()), 
       ['AIRPORT_CODE_destination', 'AIRPORT_NAME_destination', 'CITY_NAME_destination', 'COUNTRY_NAME_destination', 
        'STATE_NAME_destination', 'STATE_CODE_destination', 'LATITUDE_destination', 'LONGITUDE_destination']] = np.nan

In [34]:
# drop destination_Longitude mistachings
fw.loc[(fw.destination_Longitude < 0) & (fw.LONGITUDE_destination > 0), 
       ['AIRPORT_CODE_destination', 'AIRPORT_NAME_destination', 'CITY_NAME_destination', 'COUNTRY_NAME_destination', 
        'STATE_NAME_destination', 'STATE_CODE_destination', 'LATITUDE_destination', 'LONGITUDE_destination']] = np.nan

In [35]:
# drop city_destination mismatching
fw.loc[(fw.city_destination != fw.CITY_NAME_destination) &(fw.STATE_NAME_destination.notnull()),
       ['AIRPORT_CODE_destination', 'AIRPORT_NAME_destination', 'CITY_NAME_destination', 'COUNTRY_NAME_destination', 
        'STATE_NAME_destination', 'STATE_CODE_destination', 'LATITUDE_destination', 'LONGITUDE_destination']] = np.nan

In [36]:
# replace null values in iata_code_destination
fw.loc[fw.iata_code_destination.isna(),'iata_code_destination']  = fw.loc[fw.iata_code_destination.isna(), 'AIRPORT_CODE_destination']

In [37]:
# replace NA origin_Latitude and origin_Longitude
fw.loc[fw.origin_Latitude.isna(),'origin_Latitude'] = fw.loc[fw.origin_Latitude.isna()].origin_Latitude.fillna(0) + fw.LATITUDE_origin
fw.loc[fw.origin_Longitude.isna(), 'origin_Longitude'] = fw.loc[fw.origin_Longitude.isna()].origin_Longitude.fillna(0) + fw.LONGITUDE_origin

# replace NA destination_Latitude and destination_Longitude
fw.loc[fw.destination_Latitude.isna(),'destination_Latitude'] = fw.loc[fw.destination_Latitude.isna()].destination_Latitude.fillna(0) + fw.LATITUDE_destination
fw.loc[fw.destination_Longitude.isna(), 'destination_Longitude'] = fw.loc[fw.destination_Longitude.isna()].destination_Longitude.fillna(0) + fw.LONGITUDE_destination

In [38]:
# drop duplicated rows
fw = fw.drop(['STATE_CODE_origin', 'AIRPORT_CODE_origin', 'LATITUDE_origin', 'LONGITUDE_origin', 'city_origin'], axis=1)
fw = fw.drop(['STATE_CODE_destination', 'AIRPORT_CODE_destination', 'LATITUDE_destination', 'LONGITUDE_destination', 'city_destination'], axis=1)

In [39]:
# rearrange for convenience
fw = fw.iloc[:,:15].join([fw.iloc[:,15], fw.iloc[:,17], fw.iloc[:, 21:25], fw.iloc[:,19],
                    fw.iloc[:,16], fw.iloc[:,18], fw.iloc[:, 25:29], fw.iloc[:,20]])

In [40]:
# complete origin ICAO codes
fw = fw.merge(airport_codes.drop_duplicates(subset=['iata_code'])[['iata_code','ident']], how='left', left_on='iata_code_origin', right_on='iata_code')
fw.loc[(fw.ICAO_code_origin.isna()) & (fw.iata_code_origin.notnull()), 'ICAO_code_origin'] = fw.loc[:,'ident']
fw = fw.drop(['ident','iata_code'], axis=1)

In [41]:
# complete destination ICAO codes
fw = fw.merge(airport_codes.drop_duplicates(subset=['iata_code'])[['iata_code','ident']], how='left', left_on='iata_code_destination', right_on='iata_code')
fw.loc[(fw.ICAO_code_destination.isna()) & (fw.iata_code_destination.notnull()), 'ICAO_code_destination'] = fw.loc[:,'ident']
fw = fw.drop(['ident','iata_code'], axis=1)

In [42]:
# calculate the distance
for i in range(len(fw)):
    try:
        p1 = Point(str(fw.origin_Latitude[i]) + ' ' + str(fw.origin_Longitude[i]))
        p2 = Point(str(fw.destination_Latitude[i]) + ' ' + str(fw.destination_Longitude[i]))
    
        fw.loc[i, 'distance_mi'] = distance.distance(p1,p2).miles
    except:
        pass

In [43]:
fw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23160 entries, 0 to 23159
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tail_number               23160 non-null  object 
 1   date                      23160 non-null  object 
 2   aircraft                  22966 non-null  object 
 3   origin                    23160 non-null  object 
 4   origin_location           23160 non-null  object 
 5   destination               23159 non-null  object 
 6   destination_location      23159 non-null  object 
 7   departure                 23160 non-null  object 
 8   arrival                   22734 non-null  object 
 9   duration                  22711 non-null  object 
 10  Owner                     23160 non-null  object 
 11  origin_Latitude           22773 non-null  float64
 12  origin_Longitude          22773 non-null  float64
 13  destination_Latitude      22778 non-null  float64
 14  destin

In [44]:
# rerarrange
fw = fw.iloc[:,:10].join([fw.iloc[:,-1], fw.iloc[:,10:-1]])

In [45]:
fw.to_csv('..//Datasets/NEW/FW_with_airports.csv', index=False)